# MuMoT User Manual
## Multiscale Modelling Tool

This is the user manual for [MuMoT](https://github.com/DiODeProject/MuMoT), a software tool developed at the University of Sheffield as part of the [DiODe](http://diode.group.shef.ac.uk) project

*Current MuMoT version*: **0.0**

## Working with MuMoT

MuMoT runs inside [Jupyter notebooks](http://jupyter.org) - since you are reading this User Manual you have presumably already installed Jupyter, or are using an installation provided to you

Next, you need to install MuMoT itself; since MuMoT is a Python package it can be installed from PyPi - see the [project page](https://diodeproject.github.io/MuMoT/) for more details

To run commands in 'Code' cells it's usually easiest just to hit `Shift + Enter`; here is some basic information on [working with Jupyter notebooks](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html)

With Jupyter and MuMot both installed, you simply need to import the MuMoT package into your notebook; we recommend you refer to it as `mmt`

In [ ]:
import MuMoT.MuMoT as mmt

## Defining your first model

If everything went well with importing MuMoT you should have seen the message

``Created `%%model` as an alias for `%%latex``

MuMoT models are defined using the `%%model` keyword within a cell, and have a very simple syntax; since MuMoT works with *chemical kinetic* or *reaction kinetic* models, you simply have to describe the reactions that take place, and the rates with which they occur. For example

`A + A -> A + U: s`

is a very simple reaction, in which two particles of type $A$ interact, and one changes to type $U$, at rate $s$

These kind of reaction rules are quite general; they describe a lot of chemical reactions, as well as collective behaviour in which individuals interact to change each other's state, and even demographic models in which individuals are born and die

Our first model will be based on signalling behaviour observed in honeybee swarms, as described in the following papers:
* Seeley, T.D, Visscher, P.K. Schlegel, T., Hogan, P.M., Franks, N.R. & Marshall, J.A.R. (2012) [Stop signals provide cross inhibition in collective decision-making by honeybee swarms](http://www.sciencemag.org/content/335/6064/108.full.pdf). *Science* **335**, 108-111
* Pais, D., Hogan, P.M., Schlegel, T., Franks, N.R., Leonard, N.E. & Marshall, J.A.R. (2013) [A mechanism for value-sensitive decision-making](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216).  *PLoS one* **8**(9), e73216

The `$` at the start and end just allow us to use LaTeX [codes](http://reu.dimacs.rutgers.edu/Symbols.pdf) for our state and rate labels, allowing us to use  letters from different alphabets, such as Greek, and other nice formatting - if you don't know LaTeX don't worry about this - leave in the `$` signs, they won't hurt

**N.B.** currently it is important to put spaces between all elements of each rule, as below

OK, here we go...

In [ ]:
%%model
$
U -> A : g_A
U -> B : g_B
A -> U : a_A
B -> U : a_B
A + U -> A + A : r_A
B + U -> B + B : r_B
A + B -> A + U : s
A + B -> B + U : s
$

Now we have our model, and Jupyter should have reprinted the rules underneath the cell you input them in

You can see that actually the `_` code enables you to use subscripts, which can be handy for distinguishing related rates - in this model there are two distinct nest sites for the honeybee swarm to choose between, *A* and *B*, and rates associated with each one

We have our model, but currently it is not useful; it's only a definition, not a fully-fledged model that we can start analysing with MuMoT. To make one of those we have to use the `parseModel` command; we simply pass in a reference to the cell where we defined our model...

In [ ]:
model1 = mmt.parseModel(In[2])

If there is no output from `parseModel` that is good news - it means the model was valid. Now we have an object, which we called `model1`, which is a version of the model that we can start doing useful work with. For example, we can see its 'reactants' (or states, or types of individual; remember the terminology comes from chemical reactions)...

In [ ]:
model1.showReactants()

...and the rates the reactions happen at

In [ ]:
model1.showRates()

Note that here MuMoT also helpfully tells us which reactions in the original model definition each rate is associated with

We can also look at the reaction equations in a slightly nicer format...

In [ ]:
model1.show()

We can even see a simple figure representing our model...

In [ ]:
model1.visualise()

This figure attempts to represent the nature of the interactions between reactants graphically, from simple transitions (arrows), inhibitions (filled circles) and induced switches (arrows with filled circles at the origin); to understand the different interaction patterns try relating the arrows with the reactions we defined above, by reference to the rates they are labelled with

## Exploring your new model

If you want to know what commands you can send to an object, Jupyter lets you type out the object name, then a `.`, then hit `Tab` to see a list of accepted commands. You can also consult the [Command reference](#comreference) for a full list of commands, and details on their use

One commmand that looks intruiguing is `showODEs`

In [ ]:
model1.showODEs()

This is where things start getting interesting, because equations are what mathematicians analyse. `showODEs` shows the model's Ordinary Differential Equations, which describe how the populations of reactants in the model change over time. MuMoT has derived these equations for you automatically, from your description of the reactions. If you are familiar with ODEs then you should be able to read these quite easily. However even if you are not, you can still work with them

So, what analysis can we do in MuMoT now we've found our ODEs?

Well, we might want to look at how the state of the system (the proportions of the different reactants) change on average over time...

...but before we can do that, we need to let MuMoT know that this is actually a simpler set of equations than it appears to be. $A$, $B$, and $U$ represent states that honebees can be in during the swarming process, either committed to site *A*, committed to site *B*, or *U*ncommitted. Because there is a constant number of individuals in the swarm during a decision, one of these equations is redundant; the change in the uncommitted bee frequency can be worked out from the change in frequencies of A-committed and B-committed bees, for example

We let MuMoT know this by defining one of the reactants in terms of the other, using `substitute`

In [ ]:
model2 = model1.substitute('U = N - A - B')

Now we have a simplified model, `model2`, which only has two equations, because we have told MuMoT that the total number of uncommitted bees $U$ is the total number of bees in the swarm $N$, minus the numbers committed to the two different nests ($A$ and $B$)

In [ ]:
model2.showODEs()

Now we can look at how the model behaves, by asking for a vector plot of the two reactants $A$ and $B$...

In [ ]:
vector1 = model2.vector('A', 'B')

Now we can see how to start working with models in MuMoT. The figure you see is a vector plot, which shows on average how the state of the system (proportions of bees in states $A$ and $B$) will change over time, from any point in the state space. The arrows give the direction the system will move in, and their lengths show how fast. Because the total number of bees is constant, $A+B$ can never exceed 1, so the top right hand triangle of the figure is greyed out; these are impossible states. The number of uncommitted bees $U$, which we hid away earlier, is not shown, but it is simply $1-A-B$. But what we really care about in a decision-making process is how the number of 'voters' for the competing options changes over time

You will also notice that there are some sliders above the figure; MuMoT gives you a slider for every rate in the model, and by changing their values, you can see what happens to the behaviour of the system

Jupyter also lets you interact nicely with the figure; you can zoom in, back out, pan around, and save, all by clicking the icons that appear underneath it. If you want to save a figure for a paper, simply click the disk icon. And if while exploring the model you find a nice figure, you can click the 'off' icon in its top right to snapshot it, and generate a new interactive figure to play with. You can then simply right-click your snapshots to save them from your browser - just don't forget to note the parameters that generated them - you can find that out by looking at the logs... later in the manual we will see how to take notes properly to reproduce analyses (see [Bookmarking](#bookmarking))

Every 'controller' in MuMoT keeps a log of everything it's been asked to do; we called our controller `vector1` when we created it, so it is easy to check what it has been up to... (WARNING: if you have been playing with the sliders a lot, there could be a lot in the logs! To only see the last few entries we set the `tail` argument to `True`; if `tail` is not set it will be assumed to be `False`)

In [ ]:
vector1.showLogs(tail = True)

For papers and presentations we can produce somewhat prettier plots similar to figure 1 above, using `stream`. But before we do that, one unfortunate thing about our `vector1` controller is that there are just too many sliders. We might choose to describe all the rates (*a*bandonment, *r*ecruitment (via the waggle dance), and individual discovery and commitment (*g*)) in terms of the *v*alues of the nest sites... we can do this by using `substitute` again

In [ ]:
model3 = model2.substitute('a_A = 1/v_A, a_B = 1/v_B, g_A = v_A, g_B = v_B, r_A = v_A, r_B = v_B')

Our new model is a little simpler, with just three rates, $v_A$, $v_B$ and $s$

In [ ]:
model3.show()

Still, in decision-making what we really care about is how good options are on average, and their difference in value. Pais et al. (2013) thus defined option values in terms of deviation from the average value, so
$$\displaystyle v_A=\mu+\frac{\Delta}{2}$$
and
$$\displaystyle v_B=\mu-\frac{\Delta}{2}$$

In [ ]:
model4 = model3.substitute('v_A = \mu + \Delta/2, v_B = \mu - \Delta/2')
model4.showODEs()

This model is much easier to interact with...

In [ ]:
stream1 = model4.stream('A','B', params=[ ('\\mu', 3) ], showFixedPoints = False)

A `stream` plot is just like a `vector` plot, except now lines show the average change of the system over time in more detail, and their shading represents the speed of change, from slow (light grey) to fast (black)

Note also that here we added a *keyword argument* `showFixedPoints = True` - this can also be applied to `vector` plots, and plots the stationary points of the equation. If the stationary point is *stable* (arrows converge on that point) then it is represented with a filled circle, and if it is *unstable* (arrows leave that point) then a hollow circle is shown. Displaying fixed points can slow down plots, so by default they are not shown, and you need to explicitly request they are included

Analysis of fixed points is an important technique for understanding *dynamical systems* such as our model. But while exploring a model interactively by changing its parameters and seeing how it behaves can help develop intuition, really we need a more systematic approach. Fortunately we can do this via *bifurcation plots*, using the `bifurcation` command...

`bifurcation` takes two arguments, the first is the parameter we want to vary systematically, the second is the state we want to observe

In [ ]:
bifurcation1 = model4.bifurcation('s','A-B', params=[ ('\\mu', 3)], showInitSV=False, choose_xrange=[0,5],
                                 BifParInit=4.5)
#move Delta slider to zero -> pitchfork bifurcation
#then move Delta slider to 0.1 -> unfolding of pitchfork bifurcation

Our new bifurcation plot lets us see what happens to fixed points in the system as we vary a control parameter, in this case the cross-inhibitory stop-signalling rate between bees, $s$. We have also chosen not to observe a single reactant as our state variable, but rather the difference between two of them; this makes sense because in decision-making we care about the difference in votes for options

Our plot shows the location of fixed points, and they are either depicted as *stable* (solid line) or *unstable* (dashed line). Where the stability of fixed points changes, or fixed points appear of dissappear, these are referred to as *bifurcation points*. Using the interactive `bifurcation` plot above see if you can reproduce the bifurcation diagrams shown in figure 5(i) and 5(ii) of [Pais et al. (2013)](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216)

Figure 5(iii) of [Pais et al. (2013)](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0073216) requires us to specify a different bifurcation parameter...

In [ ]:
bifurcation2 = model4.bifurcation('\Delta','A-B',params=[ ('\\mu', 5), ('s', 5)], showInitSV=False, 
                                  choose_xrange=[-2, 2], BifParInit=2)

## The effects of finite system size

The ODE models we have been looking at based on our model are very useful, but they represent an idealised scenario; they describe the behaviour of a system with infinitely many components or, to put it another way, the expected behaviour of the system, minus any noise

Noise is very important in collective behavour systems, however, and can have a variety of causes. The first cause we will consider is noisy fluctuations from the infinite population ideal, caused by having a much smaller, finite, population. To begin analysing the noise in our system, dependent on its size, we can deploy two main techniques; stochastic simulation, and statistical physics

We will return to stochastic simulation later, concentrating first on statistical physics analysis. As you might imagine, the maths involved in a statistic physics analysis is very advanced, based on deriving and then analysing the *Master Equation*; fortunately MuMoT automates this analysis for you

Brace yourself, this might look intimidating...

### Introductory example: Production and decay of protein $P$ and its dimerization into $P_2$ (called $X$ and $Y$ below)
- see F. Hayot & C. Jayaprakash (2004), Physical Biology 1, pp. 205-210 

In [ ]:
%%model
$
\emptyset -> X : k_3
X -> \emptyset : k
X + X -> Y + \emptyset : k_1
Y + \emptyset -> X + X : k_2
$

In [ ]:
model_PQ = mmt.parseModel(In[19])

In [ ]:
model_PQ.show()

<font size="+1" > <b>Note:</b> X and Y are reactants (compounds). The amount of each compound in a given volume is represented by an absolute number (integer)</font>

<br>

<font size="+1"> Let's display the ODEs ... </font>

In [ ]:
model_PQ.showODEs()

<font size="+1" >What do X and Y represent in the ODEs?<br>  
Are these absolute numbers? If so, shouldn't $X^2$ be replaced by $X(X-1)$, for example?</font>

<br>

<font size="+1" > We need a principled way to derive the ODEs to answer that question:</font>


## The chemical Master equation

Master equations describe the time evolution of a system that, in general, can be in one of many different possible states at a given point in time with a certain probability. Over time, the system makes transitions so that the overall state changes continuously. Transitions are governed by transition probabilities that are usually summarised in a transition matrix. The Master equation is a set of first-order differential equations that describe the evolution of the state dependent probabilities driven by the transition matrix.

The <b>chemical Master equation</b> is a master equation for the probabilities that the system has any given composition of rectants as a function of time.

In [ ]:
model_PQ.showMasterEquation()

- see N. van Kampen (1981), Stochastic processes in physics and chemistry & F. Hayot & C. Jayaprakash (2004), Physical Biology 1, pp. 205-210

<font size="+1" > Here we used step operators $E_{op}$ to express the chemical Master equation in a compact form.<br>

There is only a limited number of simple cases that allow to solve the Master equation exactly. Analytical approximations may help.
</font>

## Linear noise approximation via system size expansion of the Master equation

<br>

<font size="+1" ><b>Idea:</b> Expand the Master equation in powers of a small parameter </font>

<br>
$$\sim \frac{1}{\sqrt{system\, size}}$$
<br>

<font size="+1" >Let's do the expansion with MuMoT and display it for the current model (V = system size) ... </font>

In [ ]:
model_PQ.showVanKampenExpansion()

<font size="+1" >... which are usually rather lengthy expressions. Collecting all terms $\propto \sqrt{V}$ gives ... </font>

In [ ]:
model_PQ.showODEs_vKE()

<font size="+1" >... which are the ODEs! Here $\Phi_j$ represents the concentration of reactant $j$ in the mixture. Hence, the quantities in the ODEs are concentrations or densities!</font>

<font size="+1" >If we collect all terms $\propto V^0$ we obtain ...</font>

In [ ]:
model_PQ.showFokkerPlanckEquation()

<font size="+1" >... where $\eta_X$ and $\eta_Y$ represent the noise in the system. <b>This is a linear Fokker-Planck equation whose coefficients depend on time through the concentrations $\Phi_X$ and $\Phi_Y$.</b></font>

<br>
<font size="+1" >Now we can study deterministic dynamics through the ODEs and fluctuations by means of the Fokker-Planck equation. <b>Remember:</b> ODE dynamics <b>do not</b> depend on the system size but the strength of the effect of fluctuations <b> does</b>.</font>

<br>
<font size="+1" >MuMoT can find the differential equations for first and second order moments:</font>

In [ ]:
model_PQ.showNoiseEOM()

<font size="+1" >... it tries to calculate the stationary solutions analytically.</font>

In [ ]:
model_PQ.showNoiseStationarySol()

<font size="+1" >After MuMoT threw so many formulas at you, it's good news that noise can be investigated graphically with MuMoT, too:</font>

In [ ]:
Ncorr_PQ = model_PQ.numSimNoiseCorrelations(initCondsSV = {'X': 0.2, 'Y': 0.3}, params=[('k', 1)],
                                         tend=100, tstep=0.02, tendNoise=20, 
                                         legend_loc='upper right', legend_fontsize=18)

<font size="+1" >... in addition, the effect of noise can be studied together with the fixed points of the ODE system:</font>

In [ ]:
stream_PQ = model_PQ.stream('X', 'Y', params=[('k', 1)], showFixedPoints=True, showNoise = True)

<font size="+1" >However, if you are only interested in having a quick look at the deterministic dynamics, that is easily done using:</font>

In [ ]:
evol_PQ = model_PQ.numSimStateVar(['X', 'Y'], params=[('k', 1)], 
                                  initCondsSV = {'X': 0.3}, tend=50, tstep=0.05, 
                              legend_loc='lower right', plotProportion=True)

## Another example: Population control of <em>E. coli</em> cells

- see You et al. (2004), Nature 428, pp. 868-871

<br>
<font size="+1" ><b>Reactants in the model</b>:</font>

- <font size="+1" >N   ...   <em>E. coli</em> cells the growth of which is under control</font>
- <font size="+1" >E   ...   killer protein that induces cell death</font>
- <font size="+1" >A   ...   AHL (acyl-homoserine lactone), activates the production of E</font>

<font size="+1" >In the paper by You et al. the system of differential equations is proposed, not derived. To study the system with MuMoT we need to do some reverse-engineering to find the chemical reactions that produce the ODEs in their paper!</font>

In [ ]:
%%model
$
A -> E : k_E
\emptyset + X -> X + X : k
\emptyset + X -> X + X : v_A
X + X -> X + \emptyset : k_m
E -> \emptyset : d_E
X -> A : v_A
A -> \emptyset : d_A
E + X -> E + \emptyset : d_N
A + \emptyset -> A + A : k_E
$

In [ ]:
model_Ecoli = mmt.parseModel(In[32])

In [ ]:
model_Ecoli.showODEs_vKE()

<font size="+1" >... these are exactly the equations in the You et al. paper!</font>

<br>
<font size="+1" >Let's integrate the ODEs:</font>

In [ ]:
evol_Ecoli = model_Ecoli.numSimStateVar(['A', 'E', 'X'], initCondsSV = {'A': 0.2, 'E': 0.3, 'X': 0.5},
                              tend=50, tstep=0.05, legend_loc='center right', plotProportion=False)

<font size="+1" >... and plot the noise-noise correlations:</font>

In [ ]:
Ncorr_Ecoli = model_Ecoli.numSimNoiseCorrelations(tend=100, tstep=0.02, tendNoise=20,
                                         initCondsSV = {'A': 0.2, 'E': 0.3, 'X': 0.5},
                                         legend_loc='upper right', legend_fontsize=16)

## Computational approaches

The computational approximation is achieved through the efficient and accurate *Stochastic Simulation Algorithm* (SSA) (or *Doob-Gillespie algorithm*) [(Gillespie, 1976)](#references). Alternatively, MuMot gives access to the SSA simulator to simply visualise the behaviour of the system over time.

To run the SSA simulations on your model, you can use the command `SSA()`. This command allow you to run a single simulation run, you can specify several advanced parameters from the _"Advanced options"_ tab. In particular, you can customise the following parameters:
* proportion of each reactant at timestep zero. Remember that the sum of all (non-constant) reactants must be 1.0 (type: float in range [0,1])
* simulation time is the length of each simulation run (type: float larger than 0)
* random seed which is a number to control (and repeat) random stochastic effects. Any run that uses the same random seed will have tha same identical random fluctuations (type: int in range [0, MAX_RANDOM_SEED])
* flag to plot proportions (i.e. reactants are in range [0,1]) or full populations (i.e. reactants are in range [0,systemSize])(type: boolean)
* flag to plot results in realtime. If True, the plot is updated each timestep of the simulation; if False, the plot is updated once at the end of the simulation (type: boolean)
* number of simulation runs to be executed (type: int larger than 0)
* flag to aggregate or not the results from several runs
* type of visualisation. Which can be:
  * Temporal evolution with y-axis showing the population size of each reactant varying as a function of the time on the x-axis. When aggregated, the results are visualised as boxplots. 
  * Final distribution of the population among the reactants. In this visualisation, the use can specify the reactants on x and y axes.  When aggregated, the results are visualised as an ellipse estimated as the covariance matrix
  * Barplot showing the final distribution. When aggregated, the barplots show the average and the error bars the standard deviation; otherwise the results of the last simulation run are displayed

(Note that modifying the widgets: type of visualisation, flag to plot proportions, and flag to aggregate does not trigger the computation of new data. All other widgets do.)

Try to modify the parameters and see how they change your results.

In [ ]:
ssa = model4.SSA(initWidgets={'initialState': {'U': [1,0,1,0.05], 'B': [0,0,1,0.05], 'A': [0,0,1,0.05]},
                              'systemSize':[50,5,100,1] })

## Spatial noise

MuMoT gives also access to more sophysticated stochastic simulations of your model. Through the command `multiagent()` you can run simulations either with static agents interacting over a static communication network or mobile agents communicating to neighbours within a local range of communication.

Each agent represent a reactant, it exchanges messages to interact with other agents and changes its internal state through a probabilistic finite state machine. The conversion of transition rates into the probabilities is done following the methodology proposed in:

* Reina A., Valentini G., Fernández-Oto A., Dorigo M. & Trianni V. (2015) [A design pattern for decentralised decision making](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0140950).  *PLoS one* **10**(10), e0140950

The multiagent simulations have several _Advanced options_. They have all the options described above for the SSA simulations and, in addition, you can also modify and select:
* type of network. Available types are:
  * Full graph is a complete network in which every agent communicates with every agent
  * Random network with topology Erdos-Renyi. The widgets allow to specify:
    * the linking probability with which each agent has a communication edge with another agent
  * Random network with topology Barabasi-Albert. The widgets allow to specify:
    * the number of edges every new node (during creation) is linked to
  * Simulation of moving particles in which agent move in a periodic space and interact locally. The widgets allow to specify:
    * range of interaction of the agent
    * motion correlatedness of the agent. Correlatedness equal 1 correponds to straight movement, Correlatedness equal 0 corresponds to uncorrelated random walk.
    * speed of the agent, i.e. displacement in one timestep (type: float in [0,1])
    * flag to show the motion trajectory of the agents
    * flag to show interactions, agent within communication range are liked through a line
* length of one timestep, the maximum size is determined by the rates (type: float > 0)

(Note that in addition to the one specified for the SSA simulations, here the widgets to show motion trajectory and to show interactions do not trigger the recomputation of the simulations)
Static network are available only for models that do not contains the empty-set or constant reactant, instead moving particles have no constraints; see [Supported features](#features).

Try to modify the parameters and see how they change your results.

In [ ]:
ssa = model4.multiagent(initWidgets={'initialState': {'U': [1,0,1,0.05], 'B': [0,0,1,0.05], 'A': [0,0,1,0.05]},
                              'systemSize':[20,5,100,1], 'visualisationType':'graph' })

## Bookmarking
<a id='bookmarking'></a>

If you find an interesting parameter set and want to preserve it, you can store those parameter values in a form that lets you reproduce your analysis. Simply click on the bookmark button underneath the sliders; you should see a message that a bookmark has been pasted to the logs, and instructions how to view it. Let's follow these instructions from our latest plot... click the bookmark button, then run the following command:

In [ ]:
stream1.showLogs(tail = True)

Now we need simply copy the last line of the log file, paste into a new cell, and replace `<modelName>` with the model we are working with, in this case `model5`; this is easily done by clicking in the cell then using *Find and Replace* from the *Edit* menu. We will also give our new bookmarked view a unique name, so we can refer to it if we need to in the future...

In [ ]:
bookmark1 = model2.stream('A', 'B', params = [('\\Delta', 0.5), ('s', 0.5), ('\\mu', 3), ('plotLimits', 1), 
                                              ('systemSize', 1)], showFixedPoints = False, bookmark = False)

## Supported features
<a id='features'></a>
<table>
  <tr>
    <th>Command</th>
    <th>1d-system</th> 
    <th>2d-system</th> 
    <th>3d-system</th>
    <th>empty-set</th>
    <th>constant reactants</th>
  </tr>
  <tr>
    <td><code>stream</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10005;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>vector</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>SSA</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>multiagent (static network)</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10005;</td>
    <td>&#10005;</td>
  </tr>
  <tr>
    <td><code>multiagent (moving particles)</code></td>
    <td>&#10003;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
  <tr>
    <td><code>stream (showNoise mathematically)</code></td>
    <td>&#10003;</td> 
    <td>&#10005;</td> 
    <td>&#10005;</td>
    <td>&#10005;</td>
    <td>&#10005;</td>
  </tr>
  <tr>
    <td><code>stream (showNoise computationally with SSA)</code></td>
    <td>&#10005;</td> 
    <td>&#10003;</td> 
    <td>&#10003;</td>
    <td>&#10003;</td>
    <td>&#10003;</td>
  </tr>
</table>

## Command reference
<a id='comreference'></a>
Arguments in angle brackets are optional

Optional keywords are given with the default value they will have if not supplied

* `stream(stateVariable1, stateVariable2)`

    Display interactive stream plot of `stateVariable1` (x-axis) and `stateVariable2` (y-axis)

    *Keywords:* `showFixedPoints = False`, `showNoise = False`
* `vector(stateVariable1, stateVariable2, <stateVariable3>)`

    Display interactive vector plot of `stateVariable1` (x-axis) and `stateVariable2` (y-axis), and optionally `stateVariable3` (z-axis)

    *Keywords:* `showFixedPoints = False`, `showNoise = False`

## References
<a id='references'></a>
* Gillespie, D. T. (1976). A general method for numerically simulating the stochastic time evolution of coupled chemical reactions. *Journal of Computational Physics* **22** (4): 403–434
* van Kampen, N. (2007) *Stochastic Processes in Physics and Chemistry (Third Edition)*. North-Holland
* Murray, J. D. (2002) *Mathematical Biology I. An Introduction (Second Edition)*. Springer